In [70]:
import os
import re
import sys
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import random
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.nn import Linear, ReLU, Dropout, Conv2d, MaxPool2d
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch.optim import AdamW

import gurobipy as gb
from gurobipy import GRB
import time

### Check if GPU available

In [71]:
# set CUDA_VISIBLE_DEVICES=0
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [72]:
train_test_dir = os.path.join(os.getcwd(), "dataGeneration/preprocessed_data_test")

X_test = np.load(os.path.join(train_test_dir, "X_test.npy"))
y_test = np.load(os.path.join(train_test_dir, "y_test.npy"))
index_test = np.load(os.path.join(train_test_dir, "indices_test.npy")).astype("int64")

solTime_test = np.load(os.path.join(train_test_dir, "solTime_test.npy"))
objVal_test = np.load(os.path.join(train_test_dir, "objVal_test.npy"))
schedule_test = np.load(os.path.join(train_test_dir, "schedule_test.npy")).astype("int32")
model_test = np.load(os.path.join(train_test_dir, "model_test.npy")).astype("int32")


In [73]:
X_test = np.transpose(X_test, (0,1,3,2))

print(X_test.shape)
print(X_test.dtype)
print(y_test.shape)
print(y_test.dtype)
print(index_test.shape)
print(index_test.dtype)

(200, 5, 169, 48)
float64
(200, 5, 767300)
float64
(200, 5, 767300)
int64


### Building the CNN network

In [74]:
in_channels = X_test.shape[2]
col = X_test.shape[3]

out_channels = y_test.shape[-1]
nbScen = X_test.shape[1]

print(in_channels, col, out_channels)

169 48 767300


In [75]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()

        self.hidden_ch = 64
        self.dp = 0.1

        self.feature_extractor = nn.Sequential(
            nn.Conv1d(in_channels, self.hidden_ch, 11, stride=1, padding=0),
            nn.ReLU(),
            nn.Dropout(self.dp),
            nn.MaxPool1d(5, stride=1, padding=0),

            nn.Conv1d(self.hidden_ch, self.hidden_ch*2, 7, stride=1, padding=0),
            nn.ReLU(),
            nn.Dropout(self.dp),
            nn.MaxPool1d(5, stride=1, padding=0),

            nn.Conv1d(self.hidden_ch*2, self.hidden_ch, 3, stride=1, padding=0),
            nn.ReLU(),
            nn.Dropout(self.dp),
            nn.MaxPool1d(5, stride=1, padding=0),
        )
        
        n_channels = self.feature_extractor(torch.zeros(1, in_channels, col)).size(-1)

        self.classifier = nn.Sequential(
            nn.MaxPool1d(n_channels), # GAP
            nn.Flatten(),
            nn.Linear(self.hidden_ch, self.hidden_ch*2),
            nn.ReLU(),
            nn.Dropout(self.dp),
            nn.Linear(self.hidden_ch*2, self.hidden_ch*2),
            nn.ReLU(),
            nn.Dropout(self.dp),
            nn.Linear(self.hidden_ch*2, out_channels),
            nn.Sigmoid())


    def forward(self, x):
        features = self.feature_extractor(x)
        out = self.classifier(features)
        return out
        

### Create Dataset and DataLoader

In [76]:
config = {
        'batch_size' : 8, # Num samples to average over for gradient updates
        'EPOCHS' : 500, # Num times to iterate over the entire dataset
        'LEARNING_RATE' : 5e-4, # Learning rate for the optimizer
        'WEIGHT_DECAY' : 1e-4, # Weight decay parameter for the Adam optimizer
    }

In [77]:
class coordinationDataset(TensorDataset):
    def __init__(self, X, y):
        super(coordinationDataset, self).__init__()
        self.X = X
        self.y = y
        
    def __getitem__(self, index):
        X = self.X[index]
        y = self.y[index]
        
        X_tensor = torch.tensor(X, dtype=torch.float32)
        y_tensor = torch.round(torch.tensor(y, dtype=torch.float32))

        return X_tensor, y_tensor
    
    def __len__(self):
        return len(self.X)

In [78]:
test_dataset = coordinationDataset(X_test, y_test)

print(test_dataset.X[0].shape)

(5, 169, 48)


In [79]:
net = NeuralNetwork()

batch_size = config['batch_size']
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

optimizer = optim.Adam(net.parameters(), lr=config["LEARNING_RATE"])
total_steps = len(test_loader)


#### Define custom loss function

In [80]:
def asymmetric_loss(predict, target, gamma_neg=0.3, gamma_pos=0, clip=0.0, eps=1e-8, disable_torch_grad_focal_loss=True):

    """"
    Parameters
    ----------
    x: input logits
    y: targets (multi-label binarized vector)
    """

    # Calculating Probabilities
    x_sigmoid = predict
    xs_pos = x_sigmoid
    xs_neg = 1 - x_sigmoid

    # Asymmetric Clipping
    if clip is not None and clip > 0:
        xs_neg = (xs_neg + clip).clamp(max=1)

    # Basic CE calculation
    los_pos = target * torch.log(xs_pos.clamp(min=eps))
    los_neg = (1 - target) * torch.log(xs_neg.clamp(min=eps))
    loss = los_pos + los_neg

    # Asymmetric Focusing
    if gamma_neg > 0 or gamma_pos > 0:
        if disable_torch_grad_focal_loss:
            torch.set_grad_enabled(False)
        pt0 = xs_pos * target
        pt1 = xs_neg * (1 - target)  # pt = p if t > 0 else 1-p
        pt = pt0 + pt1
        one_sided_gamma = gamma_pos * target + gamma_neg * (1 - target)
        one_sided_w = torch.pow(1 - pt, one_sided_gamma)
        if disable_torch_grad_focal_loss:
            torch.set_grad_enabled(True)
        loss *= one_sided_w

    return -loss.sum()


### Model Testing

In [81]:
# load the model
interval = 20

net = NeuralNetwork()
net.load_state_dict(torch.load(os.path.join(os.getcwd(), f"ML_Model/CNN_1D_coordination_{interval}.pth")))

C:\Users\jkang\AppData\Local\Temp\ipykernel_14048\2140383097.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load(os.path.join(os.getcwd(), f"M

<All keys matched successfully>

In [82]:
# test number of feasible solutions
# test the model on the test set
net.eval()
net.to(device)

NeuralNetwork(
  (feature_extractor): Sequential(
    (0): Conv1d(169, 64, kernel_size=(11,), stride=(1,))
    (1): ReLU()
    (2): Dropout(p=0.1, inplace=False)
    (3): MaxPool1d(kernel_size=5, stride=1, padding=0, dilation=1, ceil_mode=False)
    (4): Conv1d(64, 128, kernel_size=(7,), stride=(1,))
    (5): ReLU()
    (6): Dropout(p=0.1, inplace=False)
    (7): MaxPool1d(kernel_size=5, stride=1, padding=0, dilation=1, ceil_mode=False)
    (8): Conv1d(128, 64, kernel_size=(3,), stride=(1,))
    (9): ReLU()
    (10): Dropout(p=0.1, inplace=False)
    (11): MaxPool1d(kernel_size=5, stride=1, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): MaxPool1d(kernel_size=18, stride=18, padding=0, dilation=1, ceil_mode=False)
    (1): Flatten(start_dim=1, end_dim=-1)
    (2): Linear(in_features=64, out_features=128, bias=True)
    (3): ReLU()
    (4): Dropout(p=0.1, inplace=False)
    (5): Linear(in_features=128, out_features=128, bias=True)
    (6): ReLU()
    (7):

#### Testing of bit accuracy

In [83]:
thres = 0.5

one_accuracy = []
zero_accuracy = []
bit_accuracy = []
running_loss = 0
mean_one = []
mean_zero = []

for j, data in enumerate(test_loader):
    
    net.eval()
    inputs_all, labels_all = data

    # do a for loop to perform perdiction for each scenario
    output_append = torch.tensor([], device=device)
    gt_append = torch.tensor([], device=device)

    for x in range(nbScen):
        
        inputs, labels = inputs_all[:,x,:,:].to(device), labels_all[:,x,:,].to(device)       
        optimizer.zero_grad()
        outputs = net(inputs)

        output_append = torch.concat((output_append, outputs),dim=1)
        gt_append = torch.concat((gt_append, labels),dim=1)

    output_append = output_append.reshape(-1,)
    gt_append = gt_append.reshape(-1,)

    loss_fn = nn.BCELoss()
    loss = loss_fn(output_append, gt_append)
    running_loss += loss.item()

    # start testing
    outputs_percent = output_append
    output_append = torch.where(output_append >= thres, torch.ceil(output_append), torch.floor(output_append)).reshape(-1,)
    # outputs = torch.round(outputs)

    one_labels = torch.where(gt_append == 1)
    zero_labels = torch.where(gt_append == 0)
    
    one_outputs = output_append[one_labels]
    zero_outputs = output_append[zero_labels]

    one_acc = 1 - torch.sum(torch.abs(1 - one_outputs)) / one_outputs.shape[0] # 1 minus percentage of error
    zero_acc = 1 - torch.sum(torch.abs(0 - zero_outputs)) / zero_outputs.shape[0]
    bit_acc = 1 - torch.sum(torch.abs(output_append - gt_append)) / gt_append.shape[0]

    one_accuracy.append(one_acc.cpu().detach().numpy())
    zero_accuracy.append(zero_acc.cpu().detach().numpy())
    bit_accuracy.append(bit_acc.cpu().detach().numpy())

    # mean acc
    id_1 = torch.where(output_append == 1)
    id_0 = torch.where(output_append == 0)

    p_1 = outputs_percent[id_1]
    p_0 = outputs_percent[id_0]


    y_1 = gt_append[id_1]
    y_0 = gt_append[id_0]

    y_1_1 = torch.where(y_1 == 1)
    y_1_0 = torch.where(y_1 == 0)
    y_0_1 = torch.where(y_0 == 1)
    y_0_0 = torch.where(y_0 == 0)

    avg_1 = torch.mean(torch.cat((p_1[y_1_1], torch.ones(y_1_0[0].shape[0]).to(device) - p_1[y_1_0])))
    avg_0 = torch.mean(torch.cat((p_0[y_0_1], torch.ones(y_0_0[0].shape[0]).to(device) - p_0[y_0_0])))

    # avg_1 = torch.mean(torch.cat((p_1[y_1_1],  p_1[y_1_0])))
    # avg_0 = torch.mean(torch.cat((p_0[y_0_1], p_0[y_0_0])))

    # avg_1 = torch.mean(p_1[y_1_1])
    # avg_0 = torch.mean(p_0[y_0_1])

    # avg_1 = torch.mean(p_1[y_1_0])
    # avg_0 = torch.mean(p_0[y_0_1])

    mean_one.append(avg_1.cpu().detach().numpy())
    mean_zero.append(avg_0.cpu().detach().numpy())

print("Average one bit accuracy", np.mean(one_accuracy))
print("Average zero bit accuracy", np.mean(zero_accuracy))
print("Average bit accuracy", np.mean(bit_accuracy))
print('Loss:', running_loss / len(test_loader))
print(np.mean(mean_one), np.mean(mean_zero))

Average one bit accuracy 0.60444015
Average zero bit accuracy 0.9985646
Average bit accuracy 0.9962103
Loss: 0.01090041869902052
0.70020235 0.99563277


### Test for baseline solving speed (speed cap at 300 seconds)

In [84]:
gurobi_env = gb.Env()
gurobi_env.setParam("OutputFlag", 0)
    
# loop through all test models and calculate average optimization time
opt_time = []
opt_val = []
opt_ones = []
for i, _ in enumerate(model_test):
    
    runtime = solTime_test[i]
    obj = objVal_test[i]
    
    print("Optimization time for model ", i, ": ", runtime)
    print("Optimization Value for model ", i, ": ", obj)
    
    opt_time.append(runtime)
    opt_val.append(obj)

    nbEV = np.max(schedule_test[i][:,0]) + 1
    binary_vars = y_test[i]

    opt_ones.append(np.count_nonzero(np.round(binary_vars)))


Set parameter Username
Set parameter LicenseID to value 2592862
Academic license - for non-commercial use only - expires 2025-11-29
Optimization time for model  0 :  5242.073000192642
Optimization Value for model  0 :  1284184.37081262
Optimization time for model  1 :  27.53500008583069
Optimization Value for model  1 :  1305235.433362448
Optimization time for model  2 :  7200.074999809265
Optimization Value for model  2 :  1337602.9584201737
Optimization time for model  3 :  369.0110001564026
Optimization Value for model  3 :  1371761.5977044152
Optimization time for model  4 :  7200.516000032425
Optimization Value for model  4 :  1448389.8199385172
Optimization time for model  5 :  7200.322000026703
Optimization Value for model  5 :  1386413.5596961677
Optimization time for model  6 :  7200.254999876022
Optimization Value for model  6 :  1420292.038576436
Optimization time for model  7 :  6846.6849999427795
Optimization Value for model  7 :  1431480.6251091622
Optimization time for m

In [85]:
opt_dict = {
    "opt_time": opt_time,
    "opt_val": opt_val,
    "opt_ones": opt_ones
}

result_path = os.path.join(os.getcwd(), f"Results")
with open(os.path.join(result_path, "opt_test.pkl"), 'wb') as f:
    pickle.dump(opt_dict, f)

In [86]:
result_path = os.path.join(os.getcwd(), f"Results")
with open(os.path.join(result_path, "opt_test.pkl"), 'rb') as f:
    opt_dict = pickle.load(f)

opt_time = opt_dict["opt_time"]
opt_val = opt_dict["opt_val"]
opt_ones = opt_dict["opt_ones"]

In [87]:
# flatten opt_time
print("Average optimization time: ", np.mean(opt_time))
opt_time_baseline = np.mean(opt_time)
# flatten opt_time
print("Average optimization value: ", np.mean(opt_val))
opt_val_baseline = np.mean(opt_val)
print("Number of ones: ", opt_ones)

Average optimization time:  6051.671590007543
Average optimization value:  1417083.2321979306
Number of ones:  [7075, 7083, 9452, 17724, 21136, 20725, 20698, 21026, 20936, 21316, 21568, 21369, 21062, 9917, 22093, 22024, 21865, 22692, 22014, 22719, 23052, 23004, 19466, 23062, 8411, 22857, 22686, 23551, 23757, 23228, 24372, 23805, 24027, 24055, 24579, 10533, 24229, 24619, 24237, 24739, 21331, 25203, 25183, 25276, 22003, 25754, 9270, 26133, 26264, 25953, 26508, 26277, 26489, 26328, 26707, 26893, 27018, 10930, 27387, 27998, 27433, 27479, 27895, 27810, 23799, 27888, 28288, 28515, 9561, 28253, 28171, 28615, 29371, 29291, 29738, 29768, 29622, 29260, 29790, 9628, 29498, 30115, 26000, 30414, 29910, 30692, 30081, 31103, 30936, 31355, 9884, 31334, 31273, 31707, 31586, 31425, 31151, 31633, 32121, 32774, 33011, 10579, 27926, 33020, 33325, 33495, 32696, 33600, 33467, 33412, 33769, 28898, 7405, 12284, 11565, 12724, 13046, 13223, 13903, 14147, 14320, 14064, 14637, 7543, 14359, 14388, 15675, 15935, 136

### Calculate the optimization time for each instance in the test set if we use $\color{lightblue}\text{equality constraint}$ to find optimal solution 

#### Start testing equality constraint

In [88]:
# read in paramaeters
data_dir = os.path.join(os.getcwd(), 'systemData')
EV_routes = pd.read_csv(os.path.join(data_dir, 'EV_routes.csv')).to_numpy()
bus_params = pd.read_csv(os.path.join(data_dir, 'bus_params.csv')).to_numpy()
# EV_schedules = pd.read_csv(os.path.join(data_dir, 'EV_schedules.csv')).to_numpy().astype("int32")

nbTime, nbBus, nbRoute = 48, bus_params.shape[0], EV_routes.shape[0]
traffic = np.zeros(nbTime-1)
traffic[14:20] = 1      # from 7-10am 
traffic[32:40] = 1      # from 4-8pm

charging_station = np.squeeze(pd.read_csv(os.path.join(data_dir, 'cs_params_variable.csv')).to_numpy())
non_charging_station = np.array([i for i in range(nbBus) if i not in charging_station])
nbCS = len(charging_station)

normal_nodes =  list(charging_station) + list(range(101,108))
virtual_nodes = list(range(201,205))
congest_nodes = list(range(301,324))

always_arc, normal_arc, congest_arc = [], [], []

for r in range(nbRoute):
    if (EV_routes[r,1] in (normal_nodes+virtual_nodes)) and (EV_routes[r,2] in (normal_nodes+virtual_nodes)) and (EV_routes[r,1] != EV_routes[r,2]):
        normal_arc.append(r)
    elif (EV_routes[r,1] in (normal_nodes+virtual_nodes)) and (EV_routes[r,2] in congest_nodes):
        congest_arc.append(r)
    else:
        always_arc.append(r)


In [89]:
max_EV= 100
var_per_EV = (nbRoute*(nbTime-1) + nbCS*nbTime*2)

In [21]:
import sys
np.set_printoptions(threshold=sys.maxsize)

gurobi_env = gb.Env()
gurobi_env.setParam("OutputFlag", 0)

# loop through all test models and calculate average optimization time
opt_time_thres = []
opt_val_thres = []
opt_ones_thres = []
opt_utilized_thres = []
for i, x in enumerate(model_test):
    
    path = os.path.join(os.getcwd(), "dataGeneration/model_test")

    model = gb.read(os.path.join(path, f"coordination_{x}.mps"), env=gurobi_env)
    model.setParam("OutputFlag", 0)
    model.setParam("TimeLimit", 120*60)

    ################# timing the first code block #####################
    start = time.time()

    # deep learning prediciton
    output_append = torch.tensor([], device=device)
    gt_append = torch.tensor([], device=device)

    for s in range(nbScen):
        inputs = torch.tensor(np.expand_dims(test_dataset.X[i][s], axis=0), dtype=torch.float32) 
        inputs = inputs.to(device)    
        outputs = net(inputs) 

        # extract correct amount of binary before concat
        nbEV = np.max(schedule_test[i][:,0]) + 1
        outs = outputs[:,0:var_per_EV*nbEV]
        gts = torch.tensor(y_test[i,s,0:var_per_EV*nbEV], device=device).reshape(1,-1)

        output_append = torch.concat((output_append, outs),dim=1)
        gt_append = torch.concat((gt_append, gts),dim=1)

    output_append = output_append.reshape(-1,)
    gt_append = gt_append.reshape(-1,)

    ################## end #########################

    y_pred_binary =  (output_append).reshape(-1,).cpu().detach().numpy()
    # y_pred_binary = gt_append.reshape(-1,).cpu().detach().numpy()

    modelVars = model.getVars()

    # bin_id = []
    # ###### Build the index for the variables ######
    # for sc in range(nbScen):
    #     for k in range(nbEV):
    #         # for each EV get each attribute and order it based on EV number
    #         for r in range(nbRoute):
    #             for t in range(nbTime-1):
    #                 var = model.getVarByName(f"EVArcStatus[{sc},{k},{r},{t}]")

    #                 bin_id.append(var.index)

    #     for k in range(nbEV):
    #         for c in range(nbCS):
    #             for t in range(nbTime):
    #                 var = model.getVarByName(f"EVChargeStatus[{sc},{k},{c},{t}]")

    #                 bin_id.append(var.index)
                    
    #     for k in range(nbEV):
    #         for c in range(nbCS):
    #             for t in range(nbTime):
    #                 var = model.getVarByName(f"EVDischargeStatus[{sc},{k},{c},{t}]")

    #                 bin_id.append(var.index)
    ################## end #########################
    
    # get correct amount of index
    bin_id = index_test[i][0:var_per_EV*nbEV*nbScen].reshape(-1,)

    one_threshold = (np.mean(mean_one))
    zero_threshold = (1-np.mean(mean_zero))

    # use equality constraint from here on
    for j in range(len(y_pred_binary)):
        if (y_pred_binary[j] >= one_threshold or y_pred_binary[j] <= zero_threshold):
        # if (y_pred_binary[j] >= 0):
            modelVars[bin_id[j]].setAttr("LB", round(y_pred_binary[j]))
            modelVars[bin_id[j]].setAttr("UB", round(y_pred_binary[j]))
            
    end = time.time()
    runtime1 = end - start

    opt_ones_thres.append(np.count_nonzero(y_pred_binary >= one_threshold))
    opt_utilized_thres.append(np.count_nonzero(y_pred_binary >= one_threshold)+  np.count_nonzero(y_pred_binary <= zero_threshold))

    ################# timing the second code block #####################
    start = time.time()

    model.optimize()

    end = time.time()
    runtime2 = end - start
    runtime = runtime1 + runtime2
    ################## end #########################
    
    try:
        print("Optimization time for model ", i, ": ", model.ObjVal)
        print("Optimization value for model ", i, ": ", runtime)
        opt_time_thres.append(runtime)
        opt_val_thres.append(model.ObjVal)
    except:
        print("infeasible")

    
    model.dispose()


Set parameter Username
Set parameter LicenseID to value 2592862
Academic license - for non-commercial use only - expires 2025-11-29
infeasible
infeasible
Optimization time for model  2 :  1337603.8683535054
Optimization value for model  2 :  8.665046215057373
Optimization time for model  3 :  1371759.0463044154
Optimization value for model  3 :  18.16520357131958
infeasible
infeasible
infeasible
infeasible
Optimization time for model  8 :  1382461.781913377
Optimization value for model  8 :  18.02458643913269
infeasible
Optimization time for model  10 :  1410088.9767065076
Optimization value for model  10 :  17.957112073898315
Optimization time for model  11 :  1413940.279989934
Optimization value for model  11 :  18.354421138763428
infeasible
Optimization time for model  13 :  1271838.1610787255
Optimization value for model  13 :  8.04884672164917
infeasible
infeasible
infeasible
infeasible
Optimization time for model  18 :  1483637.9799718051
Optimization value for model  18 :  18.39

In [22]:
opt_dict_thres = {
    "opt_time": opt_time_thres,
    "opt_val": opt_val_thres,
    "opt_ones": opt_ones_thres,
    "opt_utilized": opt_utilized_thres
}

result_path = os.path.join(os.getcwd(), f"Results")
with open(os.path.join(result_path, f"CNN_1D_{interval}_test_opt_thres.pkl"), 'wb') as f:
    pickle.dump(opt_dict_thres, f)

In [19]:
with open(os.path.join(result_path, f"CNN_1D_{interval}_test_opt_thres.pkl"), 'rb') as f:
    opt_dict_thres = pickle.load(f)

opt_time_thres = opt_dict_thres["opt_time"]
opt_val_thres = opt_dict_thres["opt_val"]
opt_ones_thres = opt_dict_thres["opt_ones"]
opt_utilized_thres = opt_dict_thres["opt_utilized"]

In [20]:
# flatten opt_time
opt_time_thres_mean = np.mean(opt_time_thres)
print("Average optimization time: ", opt_time_thres_mean)
# flatten opt_time
opt_val_thres_mean = np.mean(opt_val_thres)
print("Average optimization value: ", np.mean(opt_val_thres))
print("Number of feasible model", len(opt_time_thres))
print("Number of ones: ", opt_ones_thres)
print("Number of variable utilized ", opt_utilized_thres)

Average optimization time:  20.867736981131813
Average optimization value:  1424707.1059614248
Number of feasible model 165
Number of ones:  [2982, 3166, 4449, 9494, 11586, 9380, 11748, 9035, 9768, 9529, 10079, 9251, 11135, 4589, 10719, 11116, 9292, 9270, 9710, 13013, 11518, 9966, 10123, 12090, 3729, 11628, 11705, 10758, 13329, 10545, 12101, 10948, 11596, 12284, 12200, 5123, 11195, 10777, 11542, 11008, 11719, 12464, 12941, 12590, 12394, 11015, 5273, 15012, 12392, 12362, 11268, 11833, 11940, 12020, 13274, 14535, 12293, 5278, 13610, 15823, 15287, 13482, 13021, 15671, 13316, 15400, 15373, 15986, 5227, 14580, 11875, 15504, 16217, 13637, 16386, 15823, 13958, 14867, 16368, 6361, 16058, 16328, 12139, 15585, 13809, 14726, 14675, 15478, 17243, 17316, 5740, 13367, 17370, 15095, 17755, 17499, 15263, 18210, 17357, 14353, 13853, 5184, 14319, 15928, 16084, 15903, 14095, 15767, 17473, 18994, 18454, 13792, 3074, 6266, 6184, 5709, 6829, 7860, 5358, 6546, 6578, 8007, 5230, 2664, 6519, 6460, 8951, 7625, 

### Testing equality constraint with feasibility check

In [25]:
# # read in paramaeters
# data_dir = os.path.join(os.getcwd(), 'systemData')
# EV_routes = pd.read_csv(os.path.join(data_dir, 'EV_routes.csv')).to_numpy()
# bus_params = pd.read_csv(os.path.join(data_dir, 'bus_params.csv')).to_numpy()
# # EV_schedules = pd.read_csv(os.path.join(data_dir, 'EV_schedules.csv')).to_numpy().astype("int32")

# nbScen, nbTime, nbEV, nbBus, nbRoute = 5, 48, 5, bus_params.shape[0], EV_routes.shape[0]
# traffic = np.zeros(nbTime-1)
# traffic[14:20] = 1      # from 7-10am 
# traffic[32:40] = 1      # from 4-8pm

# charging_station = [4, 8, 14, 20, 23, 28]
# non_charging_station = np.array([i for i in range(nbBus) if i not in charging_station])
# nbCS = len(charging_station)

# normal_nodes =  charging_station + list(range(101,108))
# virtual_nodes = list(range(201,205))
# congest_nodes = list(range(301,324))

# always_arc, normal_arc, congest_arc = [], [], []

# for r in range(nbRoute):
#     if (EV_routes[r,1] in (normal_nodes+virtual_nodes)) and (EV_routes[r,2] in (normal_nodes+virtual_nodes)) and (EV_routes[r,1] != EV_routes[r,2]):
#         normal_arc.append(r)
#     elif (EV_routes[r,1] in (normal_nodes+virtual_nodes)) and (EV_routes[r,2] in congest_nodes):
#         congest_arc.append(r)
#     else:
#         always_arc.append(r)


In [26]:
def solution_check(y_pred_binary, model, index_test, EV_schedules):

    variables = model.getVars()
    nbEV = np.max(EV_schedules[:,0]) + 1

    EV_schedules = EV_schedules[0:nbEV*4,:]

    # ========================================= initialise parameters ===================================================

    EVArcStatus = np.zeros((nbScen, nbEV, nbRoute, nbTime-1))
    EVArcStatusIndex = np.zeros((nbScen, nbEV, nbRoute, nbTime-1))
    EVChargeStatus = np.zeros((nbScen, nbEV, nbCS, nbTime))
    EVChargeStatusIndex = np.zeros((nbScen, nbEV, nbCS, nbTime))
    EVDischargeStatus = np.zeros((nbScen, nbEV, nbCS, nbTime))
    EVDischargeStatusIndex = np.zeros((nbScen, nbEV, nbCS, nbTime))

    id_count = 0
    for sc in range(nbScen):
        for k in range(nbEV):
            # for each EV get each attribute and order it based on EV number
            for r in range(nbRoute):
                for t in range(nbTime-1):
                    EVArcStatus[sc,k,r,t] = y_pred_binary[id_count]
                    EVArcStatusIndex[sc,k,r,t] = y_pred_binary[id_count]
                    id_count = id_count + 1

            for c in range(nbCS):
                for t in range(nbTime):
                    EVChargeStatus[sc,k,c,t] = y_pred_binary[id_count]
                    EVChargeStatusIndex[sc,k,c,t] = y_pred_binary[id_count]
                    id_count = id_count + 1

            for c in range(nbCS):
                for t in range(nbTime):
                    EVDischargeStatus[sc,k,c,t] = y_pred_binary[id_count]
                    EVDischargeStatusIndex[sc,k,c,t] = y_pred_binary[id_count]
                    id_count = id_count + 1

    # corrected_y = y_pred_binary

    # ========================================= checking Arc constestts ===================================================
    # set disable arc to 0 and check activated arc
    for sc in range(nbScen):
        for s in range(nbTime-1):
            
            activate_arc = (always_arc + congest_arc) if traffic[s] else (always_arc + normal_arc)
            disable_arc = normal_arc if traffic[s] else congest_arc
            
            EVArcStatus[sc,:,np.array(disable_arc),s] = 0

        # check for EVArcStatus sum == 1
        MoveStatus = np.sum(np.round(np.clip(EVArcStatus[sc], 0, None)), axis=1)
        # print(MoveStatus)
        for i in range(MoveStatus.shape[0]):
            for j in range(MoveStatus.shape[1]):

                activate_arc = (always_arc + congest_arc) if traffic[j] else (always_arc + normal_arc)

                if MoveStatus[i,j] == 0:
                    EVArcStatus[sc,i,activate_arc,j] = -1
                elif MoveStatus[i,j] > 1:
                    EVArcStatus[sc,i,:,j] = -1
                    # max = np.argmax(EVArcStatus[i,:,j])
                    # EVArcStatus[i,:,j] = 0
                        # EVArcStatus[i,max,j] = 1

        MoveStatus = np.sum(np.round(np.clip(EVArcStatus[sc], 0, None)), axis=1)
        # print(MoveStatus)

        # check EVArc Continuity
        for k in range(nbEV):
            for s in range(nbTime-2):
                from_arc = np.max(np.round(EVArcStatus[sc,k,:,s]))
                to_arc = np.max(np.round(EVArcStatus[sc,k,:,s+1]))

                if from_arc == 1 and to_arc == 1:
                    from_node = np.argmax(np.round(EVArcStatus[sc,k,:,s]))
                    to_node = np.argmax(np.round(EVArcStatus[sc,k,:,s+1]))

                    activate_arc = (always_arc + congest_arc) if traffic[s] else (always_arc + normal_arc)
                    
                    if EV_routes[from_node, 2] != EV_routes[to_node, 1]:
                        EVArcStatus[sc,k,:,s] = -1
                        EVArcStatus[sc,k,:,s+1] = -1
                else:
                    EVArcStatus[sc,k,:,s] = -1
                    EVArcStatus[sc,k,:,s+1] = -1

        # check schedule
        nbSchedule = EV_schedules.shape[0]
        for k in range(nbSchedule):
            schedule = EV_schedules[k]  # [EV, destination, time]

            route_index = (EV_routes[:,1]==schedule[1]).nonzero()[0] if schedule[2] == 0 else (EV_routes[:,2]==schedule[1]).nonzero()[0]
            time = 0 if schedule[2] == 0 else schedule[2]-1

            activate_arc = (always_arc + congest_arc) if traffic[time] else (always_arc + normal_arc)
            disable_arc = normal_arc if traffic[time] else congest_arc

            MoveStatus = np.sum(np.round(np.clip(EVArcStatus[sc,schedule[0], route_index, time], 0, None)))
            if MoveStatus == 0:
                EVArcStatus[sc,schedule[0],activate_arc,time] = 0
                EVArcStatus[sc,schedule[0],route_index,time] = -1
            elif MoveStatus > 1:
                EVArcStatus[sc,schedule[0],activate_arc,time] = -1
                # max = np.argmax(EVArcStatus[schedule[0],route_index,time])
                # EVArcStatus[schedule[0],:,time] = 0
                # EVArcStatus[schedule[0],max,time] = 1

    # ========================================= checking charge constestts ===================================================
        stationary_index = np.array((EV_routes[:,1] == EV_routes[:,2]).nonzero()[0])
        charging_index = np.array([i for i, e in enumerate(EV_routes[:,1]) if e in charging_station])
        charge_index = [i for i, e in enumerate(stationary_index) if e in charging_index]

        for k in range(nbEV):
            for i, ii in enumerate(stationary_index[charge_index]):
                for s in range(nbTime-1):
                    if not ((np.round(np.clip(EVChargeStatus[sc,k,i,s+1], 0, None)) + np.round(np.clip(EVDischargeStatus[sc,k,i,s+1], 0, None))) <= np.round(np.clip(EVArcStatus[sc,k,ii,s], 0, None))):
                        EVChargeStatus[sc,k,i,s+1] = -1
                        EVDischargeStatus[sc,k,i,s+1] = -1

                        activate_arc = (always_arc + congest_arc) if traffic[s] else (always_arc + normal_arc)
                        EVArcStatus[sc,k,:,s] = -1

    ##### replacing y_predict ######
    corrected_y = []

    for sc in range(nbScen):
        for k in range(nbEV):
            # for each EV get each attribute and order it based on EV number
            for r in range(nbRoute):
                for t in range(nbTime-1):
                    corrected_y.append(EVArcStatus[sc,k,r,t])

            for c in range(nbCS):
                for t in range(nbTime):
                    corrected_y.append(EVChargeStatus[sc,k,c,t])

            for c in range(nbCS):
                for t in range(nbTime):
                    corrected_y.append(EVDischargeStatus[sc,k,c,t])

    return corrected_y


In [21]:
import sys
np.set_printoptions(threshold=sys.maxsize)

gurobi_env = gb.Env()
gurobi_env.setParam("OutputFlag", 0)

# loop through all test models and calculate average optimization time
opt_time_feas = []
opt_val_feas = []
opt_ones_feas = []
opt_utilized_feas = []
for i, x in enumerate(model_test):
    # if i == 28:

    path = os.path.join(os.getcwd(), "dataGeneration/model_test")

    model = gb.read(os.path.join(path, f"coordination_{x}.mps"), env=gurobi_env)
    model.setParam("OutputFlag", 0)
    model.setParam("TimeLimit", 120*60)

    ################# timing the first code block #####################
    start = time.time()

    # deep learning prediciton
    output_append = torch.tensor([], device=device)
    gt_append = torch.tensor([], device=device)

    for s in range(nbScen):
        inputs = torch.tensor(np.expand_dims(test_dataset.X[i][s], axis=0), dtype=torch.float32) 
        inputs = inputs.to(device)    
        outputs = net(inputs) 

        # extract correct amount of binary before concat
        nbEV = np.max(schedule_test[i][:,0]) + 1
        outs = outputs[:,0:var_per_EV*nbEV]
        gts = torch.tensor(y_test[i,s,0:var_per_EV*nbEV], device=device).reshape(1,-1)

        output_append = torch.concat((output_append, outs),dim=1)
        gt_append = torch.concat((gt_append, gts),dim=1)

    output_append = output_append.reshape(-1,)
    gt_append = gt_append.reshape(-1,)

    ################## end #########################

    y_pred_binary =  (output_append).reshape(-1,).cpu().detach().numpy()
    # y_pred_binary = gt_append.reshape(-1,).cpu().detach().numpy()

    modelVars = model.getVars()

    # get correct amount of index
    bin_id = index_test[i][0:var_per_EV*nbEV*nbScen].reshape(-1,)

    one_threshold = (np.mean(mean_one))
    zero_threshold = (1-np.mean(mean_zero))

    # use equality constraint from here on
    for j in range(len(y_pred_binary)):
        if (y_pred_binary[j] >= one_threshold or y_pred_binary[j] <= zero_threshold):
            modelVars[bin_id[j]].setAttr("LB", round(y_pred_binary[j]))
            modelVars[bin_id[j]].setAttr("UB", round(y_pred_binary[j]))
            
    end = time.time()
    runtime1 = end - start

    opt_ones_feas.append(np.count_nonzero(y_pred_binary >= one_threshold))
    opt_utilized_feas.append(np.count_nonzero(y_pred_binary >= one_threshold)+  np.count_nonzero(y_pred_binary <= zero_threshold))

    ################# timing the second code block #####################
    start = time.time()

    model.optimize()

    end = time.time()
    runtime2 = end - start
    runtime = runtime1 + runtime2
    ################## end #########################
    
    try:
        print("Optimization time for model ", i, ": ", model.ObjVal)
        print("Optimization value for model ", i, ": ", runtime)
        opt_time_feas.append(runtime)
        opt_val_feas.append(model.ObjVal) 

    except:
        print("infeasible, reoptimising")

        model.dispose()

        ###### reset model and re-fix the values
        model = gb.read(os.path.join(path, f"coordination_{x}.mps"), env=gurobi_env)
        model.setParam("OutputFlag", 0)
        model.setParam("TimeLimit", 120*60)

        ################# timing the code block #####################
        start = time.time()

        modelVars = model.getVars()

        # get correct amount of index
        bin_id = index_test[i][0:var_per_EV*nbEV*nbScen].reshape(-1,)

        one_threshold = (np.mean(mean_one)) + 0.1
        zero_threshold = (1-np.mean(mean_zero))

        # use equality constraint from here on
        for j in range(len(y_pred_binary)):
            if (y_pred_binary[j] >= one_threshold or y_pred_binary[j] <= zero_threshold):
                modelVars[bin_id[j]].setAttr("LB", round(y_pred_binary[j]))
                modelVars[bin_id[j]].setAttr("UB", round(y_pred_binary[j]))

        # y_pred_binary = np.where((y_pred_binary >= zero_threshold) & (y_pred_binary <= one_threshold), -1, np.round(y_pred_binary))
        # y_pred_binary = solution_check(y_pred_binary, model, bin_id, schedule_test[i])

        # # use equality constraint from here on
        # for j in range(len(y_pred_binary)):
        #     # if (y_pred_binary[j] >= one_threshold or y_pred_binary[j] <= zero_threshold):
        #     if (y_pred_binary[j] >= 0):
        #         modelVars[bin_id[j]].setAttr("LB", round(y_pred_binary[j]))
        #         modelVars[bin_id[j]].setAttr("UB", round(y_pred_binary[j]))
                
        end = time.time()
        runtime3 = end - start
        
        opt_ones_feas.append(np.count_nonzero(y_pred_binary >= one_threshold))
        opt_utilized_feas.append(np.count_nonzero(y_pred_binary >= one_threshold)+  np.count_nonzero(y_pred_binary <= zero_threshold))

        ################# timing the second code block #####################
        start = time.time()

        model.optimize()

        end = time.time()
        runtime4 = end - start
        runtime = runtime + runtime3 + runtime4 

        try:
            print("Optimization time for model ", i, ": ", model.ObjVal)
            print("Optimization value for model ", i, ": ", runtime)
            opt_time_feas.append(runtime)
            opt_val_feas.append(model.ObjVal) 
        except:
            print('infeasible')
            opt_time_feas.append(0)
            opt_val_feas.append(0) 

    
    model.dispose()
    # break
    # else:
    #     continue



Set parameter Username
Set parameter LicenseID to value 2592862
Academic license - for non-commercial use only - expires 2025-11-29
infeasible, reoptimising
infeasible
infeasible, reoptimising
infeasible
Optimization time for model  2 :  1337603.8683535054
Optimization value for model  2 :  9.122727632522583
Optimization time for model  3 :  1371759.0463044154
Optimization value for model  3 :  17.928706407546997
infeasible, reoptimising
infeasible
infeasible, reoptimising
Optimization time for model  5 :  1386413.351296174
Optimization value for model  5 :  32.38761234283447
infeasible, reoptimising
infeasible
infeasible, reoptimising
infeasible
Optimization time for model  8 :  1382461.781913377
Optimization value for model  8 :  18.357588052749634
infeasible, reoptimising
infeasible
Optimization time for model  10 :  1410088.9767065076
Optimization value for model  10 :  18.365204095840454
Optimization time for model  11 :  1413940.279989934
Optimization value for model  11 :  18.83

In [22]:
opt_dict_feas = {
    "opt_time": opt_time_feas,
    "opt_val": opt_val_feas,
    "opt_ones": opt_ones_feas,
    "opt_utilized": opt_utilized_feas
}

result_path = os.path.join(os.getcwd(), f"Results")
with open(os.path.join(result_path, f"CNN_1D_{interval}_test_opt_feas.pkl"), 'wb') as f:
    pickle.dump(opt_dict_feas, f)

In [90]:
with open(os.path.join(result_path, f"CNN_1D_{interval}_test_opt_feas.pkl"), 'rb') as f:
    opt_dict_feas = pickle.load(f)

opt_time_feas = opt_dict_feas["opt_time"]
opt_val_feas = opt_dict_feas["opt_val"]
opt_ones_feas = opt_dict_feas["opt_ones"]
opt_utilized_feas = opt_dict_feas["opt_utilized"]

In [91]:
# flatten opt_time
opt_time_feas_mean = np.mean(opt_time_feas)
print("Average optimization time: ", np.mean(opt_time_feas))
# flatten opt_time
opt_val_feas_mean = np.mean(opt_val_feas)
print("Average optimization value: ", np.mean(opt_val_feas))
print("Number of feasible model", len(opt_time_feas))
print("Number of ones: ", opt_ones_feas)
print("Number of variable utilized ", opt_utilized_feas)

Average optimization time:  20.003699326515196
Average optimization value:  1096831.135578554
Number of feasible model 200
Number of ones:  [3401, 2719, 3548, 2812, 3250, 10038, 11866, 9839, 11931, 9560, 13477, 11133, 9888, 7638, 10040, 9860, 7892, 11720, 9829, 11664, 9280, 3820, 11429, 9117, 12329, 10294, 9771, 7581, 9103, 7322, 10662, 13538, 11115, 10445, 9965, 10960, 12608, 3742, 10628, 11570, 11404, 13659, 10996, 9705, 10810, 8221, 10491, 12417, 9805, 12311, 12352, 3756, 11697, 11275, 11434, 11073, 11476, 12828, 14137, 11245, 14720, 14157, 11297, 8829, 4171, 16202, 13192, 12720, 12772, 12605, 12465, 12449, 12994, 15771, 13038, 15157, 13646, 3792, 14163, 11003, 17679, 14385, 16041, 13499, 16270, 13264, 13817, 10921, 15729, 12654, 15063, 16362, 13520, 16366, 13494, 18050, 14730, 4796, 15954, 12950, 9965, 15634, 17027, 13638, 13226, 18400, 16867, 15142, 11798, 16667, 16636, 5630, 17658, 16633, 14431, 11005, 17664, 13891, 14496, 11368, 15643, 15581, 18273, 14321, 18369, 19534, 5143, 39

### Calculate time sped up by and optimality difference

In [21]:
speedup_time_thres = 1 - (opt_time_thres_mean/opt_time_baseline)
optimality_loss_thres = (opt_val_thres_mean - opt_val_baseline) / opt_val_baseline
feasible_model_thres = len(opt_time_thres) / len(model_test) * 100
# ones_utilized_thres = np.array(opt_ones_thres) / np.array(opt_ones) *100
# prediction_utilized_thres = np.array(opt_utilized_thres) / len(index_test[0]) *100

speedup_time_feas = 1 - (opt_time_feas_mean/opt_time_baseline)
optimality_loss_feas = (opt_val_feas_mean - opt_val_baseline) / opt_val_baseline
feasible_model_feas = len(opt_time_feas) / len(model_test) * 100
# ones_utilized_feas = np.array(opt_ones_feas) / np.array(opt_ones) *100
# prediction_utilized_feas = np.array(opt_utilized_feas) / len(index_test[0]) *100

print(f"Time sped up for threshold & feasibility check: {speedup_time_thres*100} %, {speedup_time_feas*100} %")
print(f"Optimality Loss for threshold & feasibility cheack: {optimality_loss_thres*100} %, {optimality_loss_feas*100} %")
print(f"Feasible model for threshold & feasibility cheack: {feasible_model_thres} %, {feasible_model_feas} %")
# print(f"Average Number ones used for threshold & feasibility cheack: {np.mean(ones_utilized_thres)} %, {np.mean(ones_utilized_feas)} %")
# print(f"Number of prediction utilized for threshold & feasibility cheack: {np.mean(prediction_utilized_thres)} %, {np.mean(prediction_utilized_feas)} %")


Time sped up for threshold & feasibility check: 99.64259926909645 %, 99.59376415540153 %
Optimality Loss for threshold & feasibility cheack: 1.0485543958206813 %, 0.2899727780725074 %
Feasible model for threshold & feasibility cheack: 74.5 %, 95.5 %


In [92]:
opt_time_feas_arr = np.array(opt_time_feas)
opt_val_feas_arr = np.array(opt_val_feas)
opt_time_arr = np.array(opt_time)
opt_val_arr = np.array(opt_val)

inf_id = np.where(opt_time_feas_arr == 0.0)

opt_time_feas_arr[inf_id] = opt_time_arr[inf_id]
opt_val_feas_arr[inf_id] = opt_val_arr[inf_id]

spd_up = 1-(np.array(opt_time_feas_arr)/np.array(opt_time_arr))
opt_loss = ((np.array(opt_val_feas_arr) - np.array(opt_val_arr))/np.array(opt_val_arr))

print(np.mean(spd_up))
print(np.mean(opt_loss))

0.7559388059411475
3.913039588430057e-06
